In [3]:
import numpy as np
import re
import itertools

In [4]:
with open(f'input/input_day_05.txt', 'r') as file:
    raw_input = file.read().splitlines()
raw_input

['seeds: 919339981 562444630 3366006921 67827214 1496677366 101156779 4140591657 5858311 2566406753 71724353 2721360939 35899538 383860877 424668759 3649554897 442182562 2846055542 49953829 2988140126 256306471',
 '',
 'seed-to-soil map:',
 '627617777 1691901751 235673208',
 '2425244517 2483951770 157286279',
 '1339042890 1549225044 142676707',
 '481294110 381503165 89539853',
 '863290985 1007717708 39103521',
 '570833963 324719351 56783814',
 '3953140805 3714151881 155523737',
 '902394506 1941176275 61481385',
 '963875891 675869083 331848625',
 '1922840702 1046821229 502403815',
 '1481719597 1927574959 13601316',
 '1820040264 573068645 102800438',
 '1295724516 2641238049 43318374',
 '0 2002657660 481294110',
 '1495320913 0 324719351',
 '2582530796 471043018 102025627',
 '3714151881 3869675618 238988924',
 '',
 'soil-to-fertilizer map:',
 '1288462652 3191328122 309853381',
 '3216116191 1774097401 151922673',
 '1739360920 2276789875 44162492',
 '220941763 1080382821 325206789',
 '241614

In [5]:
initial_seeds = re.findall(r'\d+', raw_input[0])

In [6]:
split_value = ''
maps = [list(y) for x, y in itertools.groupby(raw_input[1:], lambda z: z == split_value) if not x]

In [32]:
maps_parsed = [
    [re.findall(r'\d+', map_) for map_ in _map[1:]] for _map in maps
]

In [9]:
def map_seed(seed, map_code_part):
    source_start = int(map_code_part[1])
    map_range = int(map_code_part[2])
    destination_start = int(map_code_part[0])
    
    if seed in range(source_start, source_start+map_range):
        difference = seed-source_start
        return destination_start + difference

In [11]:
def map_function(seed, map_code):
    destination = [
        i for i in
            [
                map_seed(seed, map_code_part)
                for map_code_part
                in map_code
            ]
        if isinstance(i, (int, float))
    ]
    return seed if not destination else destination[0]      

In [21]:
def find_final_start_zone(initial_seed):
    current_index = int(initial_seed)
    for map_code in maps_parsed:
        new_index = map_function(current_index, map_code)
        current_index = new_index
    return current_index

In [22]:
#Part 1
final_start_zone = []
for seed in initial_seeds:
    final_start_zone.append(find_final_start_zone(seed))
min(final_start_zone)

346433842

In [25]:
#Lol yeah let's check billions of cases
final_start_zone = np.inf
for seed_start, seed_range in zip(initial_seeds[0::2], initial_seeds[1::2]):
    for seed in range(int(seed_start), int(seed_start+seed_range)):
        final_start_zone = min(find_final_start_zone(seed), final_start_zone)
min(final_start_zone)

KeyboardInterrupt: 

In [ ]:
#Need inverse maps of discontinuities
def find_endpoint_inverse(map_code):
    endpoints = []
    for map_code_part in map_code:
        source_start = int(map_code_part[1])
        map_range = int(map_code_part[2])
        destination_start = int(map_code_part[0])
        
        endpoints += [source_start, 

In [64]:
def parse_endpoint_inverses(map_code):
    map_endpoints = [
        [
            int(s_start),
            int(s_start) + int(rlen) - 1
        ]
        for (d_start, s_start, rlen) 
        in map_code
    ]
    return [map_point for map_points in map_endpoints for map_point in map_points]

In [54]:
def parse_point_inverse(point, map_code):
    for (d_start, s_start, rlen) in map_code:
        if int(d_start) <= point <= int(d_start) + int(rlen) - 1:
            return point - int(d_start) + int(s_start)
    return point

In [68]:
all_inverses = [0, np.inf]
for map_code in reversed(maps_parsed):
    map_inverses = parse_endpoint_inverses(map_code)
    additional_inverses = [
        parse_point_inverse(point, map_code)
        for point
        in all_inverses
    ]
    all_inverses = map_inverses + additional_inverses

In [76]:
available_inverses = []
for seed_start, seed_range in zip(initial_seeds[0::2], initial_seeds[1::2]):
    for inverse in all_inverses:
        if int(seed_start) <= inverse <= int(seed_start)+int(seed_range)-1:
            available_inverses.append(inverse)

In [77]:
#Part 2
final_start_zone = []
for seed in available_inverses:
    final_start_zone.append(find_final_start_zone(seed))
min(final_start_zone)

60294664